# Purpose of the Code

This notebook aims to optimize the splitting of a dataset into training, development (dev), and testing sets to maximize the number of unique `Plate_NUM`s (identifiers) while also providing insights into the total records in each split. The parameters for the splits, such as the end date of training and the durations for dev and test sets, are iteratively tested to identify the best configuration.

## Key Objectives:
1. **Maximize Unique Identifiers**: Ensure the maximum number of unique `Plate_NUM`s are distributed across training, dev, and test sets.
2. **Maintain Overlap**: Ensure dev and test sets overlap sufficiently with the training set to include common identifiers.
3. **Analyze Results**: Display the top configurations with:
   - Number of unique `Plate_NUM`s in each split.
   - Total number of records in dev and test sets.

The final outcome is a configuration that balances the unique identifiers and total records across training, dev, and test subsets.

---


# Loading the Dataset

The dataset `travel_time_data.csv` contains processed historical data where `timestamp2` has been modified using a rolling window. The data will be loaded into a pandas DataFrame for analysis.

In [45]:
import pandas as pd

In [46]:
df = pd.read_csv('travel_time_data.csv')

---

# Finding the Best Date for Splitting

To determine the optimal parameters for splitting the data:
1. Iterate over various configurations of `train_end`, `dev_duration`, and `test_duration`.
2. Evaluate each configuration based on:
   - Number of unique `Plate_NUM`s in training, dev, and test subsets.
   - Total records in dev and test subsets.
3. Sort and rank configurations to identify the best balance between unique identifiers and record counts.

Below is the code implementation to achieve this.


## Code Overview:
1. **`find_optimal_split`**:
   - Iteratively splits the dataset based on varying `train_end`, `dev_duration`, and `test_duration` parameters.
   - Records the number of unique `Plate_NUM`s and total records in each split.
   - Outputs a DataFrame sorted by the total unique `Plate_NUM`s.

2. **`display_best_split`**:
   - Displays the top configurations from the results.
   - Highlights the split parameters and their corresponding results for easy comparison.

3. **Example Usage**:
   - Define the start date, maximum training days, dev duration, and test duration.
   - Use the functions to identify the best split configuration.

Below is the implementation of these functionalities.

In [47]:
# Display the top configuration and its results
def display_best_split(results_df, top_n=1):
    """
    Display the top configurations and their results.
    
    Parameters:
    results_df (DataFrame): DataFrame containing split configurations and their results.
    top_n (int): Number of top configurations to display.
    """
    # Display the top configurations
    top_results = results_df.head(top_n)
    for i, row in top_results.iterrows():
        print(f"Configuration {i + 1}:")
        print(f"Train End Date: {row['train_end'].strftime('%Y-%m-%d')}")
        print(f"Dev Duration: {row['dev_duration']} days")
        print(f"Test Duration: {row['test_duration']} days")
        print(f"Train Plates: {row['train_plates']} unique")
        print(f"Dev Plates: {row['dev_plates']} unique ({row['dev_records']} records)")
        print(f"Test Plates: {row['test_plates']} unique ({row['test_records']} records)")
        print(f"Total Unique Plates: {row['total_plates']}")
        print("-" * 50)

# Update find_optimal_split to also calculate record counts
def find_optimal_split(data, start_date, max_train_days, max_dev_days, max_test_days):
    results = []

    for train_days in range(30, max_train_days + 1, 30):  # Test train_end every 30 days
        for dev_days in range(15, max_dev_days + 1, 15):  # Test dev_duration every 15 days
            for test_days in range(15, max_test_days + 1, 15):  # Test test_duration every 15 days
                train_end = pd.to_datetime(start_date) + timedelta(days=train_days)
                dev_duration = timedelta(days=dev_days)
                test_duration = timedelta(days=test_days)

                # Perform the split
                train_data, dev_data, test_data = split_data_custom(data, train_end, dev_duration, test_duration)

                # Count unique Plate_NUMs and total records
                train_plates = len(set(train_data['Plate_NUM']))
                dev_plates = len(set(dev_data['Plate_NUM']))
                test_plates = len(set(test_data['Plate_NUM']))
                dev_records = len(dev_data)
                test_records = len(test_data)

                # Store the results
                results.append({
                    'train_end': train_end,
                    'dev_duration': dev_days,
                    'test_duration': test_days,
                    'train_plates': train_plates,
                    'dev_plates': dev_plates,
                    'test_plates': test_plates,
                    'dev_records': dev_records,
                    'test_records': test_records
                })

    # Convert to DataFrame for analysis
    results_df = pd.DataFrame(results)

    # Sort by the sum of unique Plate_NUMs across all sets (or other criteria)
    results_df['total_plates'] = results_df['train_plates'] + results_df['dev_plates'] + results_df['test_plates']
    results_df = results_df.sort_values(by='total_plates', ascending=False)

    return results_df

# Example Usage
start_date = "2015-04-13"  # Start date of the dataset
max_train_days = 150  # Maximum days to include in the training period
max_dev_days = 60  # Maximum days to include in the dev period
max_test_days = 90  # Maximum days to include in the test period

# Find the optimal split configuration
optimal_splits = find_optimal_split(data, start_date, max_train_days, max_dev_days, max_test_days)

# Display the best configuration and results
display_best_split(optimal_splits, top_n=3)


Configuration 106:
Train End Date: 2015-09-10
Dev Duration: 30 days
Test Duration: 60 days
Train Plates: 175372 unique
Dev Plates: 7559 unique (14698 records)
Test Plates: 7559 unique (15786 records)
Total Unique Plates: 190490
--------------------------------------------------
Configuration 103:
Train End Date: 2015-09-10
Dev Duration: 30 days
Test Duration: 15 days
Train Plates: 175372 unique
Dev Plates: 7559 unique (14698 records)
Test Plates: 7559 unique (15786 records)
Total Unique Plates: 190490
--------------------------------------------------
Configuration 105:
Train End Date: 2015-09-10
Dev Duration: 30 days
Test Duration: 45 days
Train Plates: 175372 unique
Dev Plates: 7559 unique (14698 records)
Test Plates: 7559 unique (15786 records)
Total Unique Plates: 190490
--------------------------------------------------


# Choosing the Best Date and Splitting the Data

Based on the analysis, select the optimal `train_end`, `dev_duration`, and `test_duration`. Split the dataset into training, dev, and test subsets using these parameters. This step ensures a balanced and meaningful division of data for further modeling or analysis.

Below is the code for the selected split.


In [48]:
# import pandas as pd
from datetime import timedelta

# Load your dataset (replace 'df' with your actual DataFrame)
data = df.copy()
data_copy = df.copy()
# Convert timestamps to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])
data_copy['timestamp'] = pd.to_datetime(data_copy['timestamp'])
# Analyze Plate_NUM counts
plate_counts = data['Plate_NUM'].value_counts()

# Filter Plate_NUMs with sufficient records
min_records_per_plate = 3  # Minimum records needed for a Plate_NUM
valid_plates = plate_counts[plate_counts >= min_records_per_plate].index
data = data[data['Plate_NUM'].isin(valid_plates)]

def split_data_custom(data, train_end, dev_duration, test_duration):
    train_data = data_copy[data_copy['timestamp'] < train_end]
    dev_data = data[(data['timestamp'] >= train_end) & (data['timestamp'] < train_end + dev_duration)]
    test_data = data[data['timestamp'] >= train_end + dev_duration]

    # Force overlap between dev and test
    train_plates = set(train_data['Plate_NUM'])
    dev_plates = set(dev_data['Plate_NUM'])
    test_plates = set(test_data['Plate_NUM'])

    # Ensure all Plate_NUMs in dev and test exist in training
    dev_plates_in_train = dev_plates.intersection(train_plates)
    test_plates_in_train = test_plates.intersection(train_plates)

    # Keep only valid Plate_NUMs in dev and test
    dev_data = dev_data[dev_data['Plate_NUM'].isin(dev_plates_in_train)]
    test_data = test_data[test_data['Plate_NUM'].isin(test_plates_in_train)]

    # Ensure overlap between dev and test
    common_plates = dev_plates.intersection(test_plates)
    dev_data = dev_data[dev_data['Plate_NUM'].isin(common_plates)]
    test_data = test_data[test_data['Plate_NUM'].isin(common_plates)]

    return train_data, dev_data, test_data

# Initial split parameters
initial_train_end = pd.Timestamp("2015-09-10")  # Starting training end date
dev_duration = timedelta(days=30)               # Dev set duration
test_duration = timedelta(days=30)              # Test set duration

# Perform the custom split
train_data, dev_data, test_data = split_data_custom(data, initial_train_end, dev_duration, test_duration)

# Print summary
print(f"Training set size: {len(train_data)}")
print(f"Dev set size: {len(dev_data)}")
print(f"Test set size: {len(test_data)}")

# Print split dates
print("\nSplit Date Ranges:")
print(f"Training set: {train_data['timestamp'].min()} to {train_data['timestamp'].max()}")
print(f"Dev set: {dev_data['timestamp'].min()} to {dev_data['timestamp'].max()}")
print(f"Test set: {test_data['timestamp'].min()} to {test_data['timestamp'].max()}")

# Check Plate_NUM distribution
train_plates = set(train_data['Plate_NUM'])
dev_plates = set(dev_data['Plate_NUM'])
test_plates = set(test_data['Plate_NUM'])

print(f"\nPlate_NUMs in training set: {len(train_plates)}")
print(f"Plate_NUMs in dev set: {len(dev_plates)}")
print(f"Plate_NUMs in test set: {len(test_plates)}")

# Validate
if not dev_plates.issubset(train_plates):
    print("Error: Dev set contains Plate_NUMs not in training set.")
if not test_plates.issubset(train_plates):
    print("Error: Test set contains Plate_NUMs not in training set.")
if not dev_plates.intersection(test_plates):
    print("Error: Dev and test sets do not overlap.")


Training set size: 208861
Dev set size: 14698
Test set size: 15786

Split Date Ranges:
Training set: 2015-04-13 12:26:07 to 2015-09-09 23:29:45
Dev set: 2015-09-10 00:06:49 to 2015-10-09 23:24:04
Test set: 2015-10-10 00:00:45 to 2016-01-19 23:12:59

Plate_NUMs in training set: 175372
Plate_NUMs in dev set: 7559
Plate_NUMs in test set: 7559


In [53]:
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)
dev_data.to_csv('dev_data.csv', index=False)